In [2]:
import json
import sympy as sp

# Polar Pulley Analytical Solutions

This notebook solves analytically for GT-2 and GT-3 pulley tooth profiles. Specifically, it solves for the locations of the relevant arc segment points and centers given the definition of the GT tooth profile. 

![Tooth Profile Diagram](diagrams/pulley_polar_eqns_diagram.svg)
*Tooth profile diagram of GT-#N tooth profiles. Specific dimensions are for a GT-2 pulley with 10 teeth*

## Solution Strategy

Solving for all of the points at once doesn't seem to complete in a reasonable amount of time, so the solution is aproached in several steps.

### Step 1

Solve for AX, AY, BX, BY, ABX, ABY, BCX, BCY. We know that *P (BCX, BCY)*, *P (ABX, ABY)*, and *P (BX, BY)* are colinear since *arc BC* is tangent to *arc AB*. We use this along with the parameter constraints and natural circular constraints to solve for the above variables. Since there are multiple solutions, we will manually evaluate and choose the correct solution.

### Step 2

Now, we need to solve for point CD, which will allow us to trivially solve for C as well as D, since they are along the lines BC -> CD and C -> O (O is the origin), and the distance along each line is defined by constants.

### Step 3

Finally, we need to solve for E, F, EF, G, FG. Firstly, we know that the angle $\alpha$ between lines O -> A and O -> G is:

> $\alpha = 2\frac{P}{PD}$

Next, the angle $\theta$ of $\overrightarrow{OB}$ with respect to $\hat{y}$ can be calculated by the formula:

> $\theta = -\sin^{-1}{\frac{BX}{\sqrt{BX^2+BY^2}}}$  

Since the angle of $\overrightarrow{OA}$ with respect to $\hat{y}$ is opposite $\overrightarrow{OB}$, then: 

> $\angle GO\hat{y} = -\angle BO\hat{y} - \alpha$  

E, F, EF, G, and FG are reflected in polar coordinates over the angle bisector of $\angle BOG$.

In [71]:
# Variables
AX, AY, BX, BY, ABX, ABY, BCX, BCY, RBXY, CX, CY, CDX, CDY, DX, DY, EX, EY, FX, FY, EFX, EFY, GX, GY, FGX, FGY = sp.symbols(
    'AX, AY, BX, BY, ABX, ABY, BCX, BCY, RBXY, CX, CY, CDX, CDY, DX, DY, EX, EY, FX, FY, EFX, EFY, GX, GY, FGX, FGY', real=True)

In [99]:
# Parameters
RAB, RBC, RCD, b, h, PLD, t, PD, P = sp.symbols(
    'RAB, RBC, RCD, b, h, PLD, t, PD, P')
params = [RAB, RBC, RCD, b, h, PLD, t, PD,P]
vals = [0.555, 1, 0.15, 0.4, 0.75, 0.254,
        10, 6.36619772368, 2]
param_associations = list(zip(params, vals))

In [12]:
# Substitutions
L = sp.symbols("L")
L_sub = PD / 2 - PLD - h + RAB

In [13]:
# Equation set 1. The goal is to solve for BCX, BCY, BX, and BY.
equations_1 = [
    AX + BX,
    AY - BY,
    ABX,
    L - ABY,
    RBXY * sp.sin(2 * b / PD) + BCX,
    RBXY * sp.cos(2 * b / PD) - BCY,
    sp.sqrt(BCX**2 + BCY**2) - RBXY,
    (BX - ABX)**2 + (BY - ABY)**2 - RAB**2,
    (BX - BCX)**2 + (BY - BCY)**2 - RBC**2,
    (BCY - ABY) / (BCX - ABX) - (ABY - BY) / (ABX - BX)
]

In [14]:
res_1 = sp.solve(equations_1, AX, AY, BX, BY, ABX,
                 ABY, BCX, BCY, RBXY, dict=True)

In [62]:
res_1_sol = res_1[1] # Hand-picked solution which matches the drawing
# Substitute back L
for key, val in res_1_sol.items():
    res_1_sol[key] = val.subs(L, L_sub)

In [63]:
equations_2 = [
    (CDX - BCX)**2 + (CDY - BCY)**2 - (RBC+RCD)**2,
    CDX**2 + CDY**2 - (PD/2 - PLD - RCD)**2
]

In [64]:
res_2 = sp.solve(equations_2, CDX, CDY, dict=True)

In [66]:
res_2_sol = res_2[0] # Hand-picked solution which matches the drawing

### Step 3

Final solution is being prepared. The equations ought to be evaluated in order and substituted sequentially to achieve efficient execution.

In [165]:
def reflect_point(x, y, theta):
    R = sp.sqrt(x**2 + y**2)
    theta_new = 2*theta + sp.asin(x/sp.sqrt(x**2 + y**2))
    x_new = -R * sp.sin(theta_new)
    y_new = R * sp.cos(theta_new)
    return x_new, y_new

In [166]:
final_sol = [(k, v) for k, v in res_1_sol.items()] + [(k, v) for k, v in res_2_sol.items()]

final_sol.append((CX, CDX-(RCD)/(RCD+RBC)*(CDX - BCX)))
final_sol.append((CY, CDY-(RCD)/(RCD+RBC)*(CDY - BCY)))

CDR, TANG = sp.symbols('CDR, TANG') # Temporary variables to make calculations quicker when ported

final_sol.append((CDR, sp.sqrt(CDX**2+CDY**2)))
final_sol.append((DX, CDX*(CDR + RCD)/(CDR)))
final_sol.append((DY, CDY*(CDR + RCD)/(CDR)))
final_sol.append((TANG, (-sp.asin(BX/(sp.sqrt(BX**2+BY**2))) + (sp.asin(BX/(sp.sqrt(BX**2+BY**2))) - 2*P/PD))/2))

EX_, EY_ = reflect_point(DX, DY, TANG)
final_sol.append((EX, EX_))
final_sol.append((EY, EY_))

FX_, FY_ = reflect_point(CX, CY, TANG)
final_sol.append((FX, FX_))
final_sol.append((FY, FY_))

EFX_, EFY_ = reflect_point(CDX, CDY, TANG)
final_sol.append((EFX, EFX_))
final_sol.append((EFY, EFY_))

GX_, GY_ = reflect_point(BX, BY, TANG)
final_sol.append((GX, GX_))
final_sol.append((GY, GY_))

FGX_, FGY_ = reflect_point(BCX, BCY, TANG)
final_sol.append((FGX, FGX_))
final_sol.append((FGY, FGY_))

In [173]:
import copy
final_sol_eval = copy.deepcopy(final_sol)

# Evaluate Solution
for i, el in enumerate(final_sol_eval):
    final_sol_eval[i] = (el[0], el[1].subs(param_associations).subs(final_sol_eval[:i]))
final_sol_eval

[(AX, -0.468387108234888),
 (AY, 2.43637664762118),
 (BX, 0.468387108234888),
 (BY, 2.43637664762118),
 (ABX, 0),
 (ABY, 2.73409886184000),
 (BCX, -0.375553627323469),
 (BCY, 2.97281306963707),
 (RBXY, 2.99644090113602),
 (CDX, 0.736474689773917),
 (CDY, 2.67973795644328),
 (CX, 0.591427517978606),
 (CY, 2.71796514512073),
 (CDR, 2.77909886184000),
 (DX, 0.776225417962459),
 (DY, 2.82437501811310),
 (TANG, -0.314159265358773),
 (EX, 1.03214642799161),
 (EY, 2.74122124127487),
 (FX, 1.11910491556020),
 (FY, 2.54651236538967),
 (EFX, 0.979289910850538),
 (EFY, 2.60084250857039),
 (GX, 1.05313313199241),
 (GY, 2.24638114720862),
 (FGX, 2.05120494695793),
 (FGY, 2.18431141085152)]

In [172]:
final_sol

[(AX,
  (-2*RAB**2*(PD/2 - PLD + RAB - h)*sin(b/PD)**2 + RAB**2*(PD/2 - PLD + RAB - h) + RAB**2*sqrt(RAB**2 - 2*RAB*RBC + RBC**2 + 4*(PD/2 - PLD + RAB - h)**2*sin(b/PD)**4 - 4*(PD/2 - PLD + RAB - h)**2*sin(b/PD)**2) - 2*RAB*RBC*(PD/2 - PLD + RAB - h)*sin(b/PD)**2 + RAB*RBC*(PD/2 - PLD + RAB - h) + RAB*RBC*sqrt(RAB**2 - 2*RAB*RBC + RBC**2 + 4*(PD/2 - PLD + RAB - h)**2*sin(b/PD)**4 - 4*(PD/2 - PLD + RAB - h)**2*sin(b/PD)**2) + 256*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**18 - 1152*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**16 + 2112*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**14 + 256*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**12*cos(b/PD)**6 - 2016*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**12 - 384*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**10*cos(b/PD)**6 + 1056*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**10 + 192*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**8*cos(b/PD)**6 - 288*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**8 - 32*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**6*cos(b/PD)**6 + 32*(PD/2 - PLD + RAB - h)**3*sin(b/PD)**6)*sin(2*b/PD)/(RAB